In [1]:
import numpy as np
import pandas as pd
import json
import glob
import os
import pprint
import re
from bs4 import BeautifulSoup 

from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.sparse import ElasticsearchRetriever

11/12/2020 09:39:54 - INFO - faiss -   Loading faiss.


In [1]:
import pandas as pd

In [2]:
clinvar_data = pd.read_csv('../data/nf_mutation_info.csv')

In [3]:
clinvar_data.head()

,Name,Gene(s),Protein change,diagnosis,Clinical significance,Last_review_date,GRCh37Chromosome,GRCh38Chromosome,VariationID
0,NM_001042492.3(NF1):c.58C>G (p.Gln20Glu),NF1,Q20E,"Neurofibromatosis, type 1",Uncertain significance,"Nov 8, 2019",17.0,17.0,955226
1,NM_000267.3(NF1):c.58C>T (p.Gln20Ter),NF1,Q20*,"Neurofibromatosis, type 1",Pathogenic,"Jan 7, 2019",17.0,17.0,576465
2,NM_000267.3(NF1):c.59A>C (p.Gln20Pro),NF1,Q20P,"Neurofibromatosis, type 1",Uncertain significance,"Aug 2, 2018",17.0,17.0,650986
3,NM_001042492.3(NF1):c.60G>C (p.Gln20His),NF1,Q20H,"Neurofibromatosis, type 1",Uncertain significance,"Oct 7, 2019",17.0,17.0,965727
4,NM_000267.3(NF1):c.60+1G>C,NF1,NaN,"Neurofibromatosis, type 1",Likely pathogenic,"Aug 28, 2018",17.0,17.0,663201


## Define the QA model

In [7]:
# free up space if needed
#!curl -X PUT -H "Content-Type: application/json" http://127.0.0.1:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}'

In [2]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

11/12/2020 09:40:02 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.453s]
11/12/2020 09:40:02 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.170s]


In [8]:
document_store.delete_all_documents(index = 'document')

10/11/2020 22:02:48 - INFO - elasticsearch -   POST http://localhost:9200/document/_delete_by_query [status:200 request:0.693s]


In [3]:
document_store.get_document_count()

11/12/2020 09:40:06 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.133s]


0

In [22]:
retriever = ElasticsearchRetriever(document_store=document_store)

In [23]:
# use the pretrained bert model on the biology pubmed dataset
reader = FARMReader("ktrapeznikov/biobert_v1.1_pubmed_squad_v2", use_gpu=False)

11/10/2020 17:39:13 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/10/2020 17:39:13 - INFO - farm.infer -   Could not find `ktrapeznikov/biobert_v1.1_pubmed_squad_v2` locally. Try to download from model hub ...
11/10/2020 17:39:17 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
11/10/2020 17:39:24 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/10/2020 17:39:24 - INFO - farm.infer -   Got ya 7 parallel workers to do inference ...
11/10/2020 17:39:24 - INFO - farm.infer -    0    0    0    0    0    0    0 
11/10/2020 17:39:24 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\
11/10/2020 17:39:24 - INFO - farm.infer -   /'\  / \  /'\  /'\  / \  / \  /'\
11/10

In [33]:
reader.save("../model/")

11/10/2020 19:20:44 - INFO - haystack.reader.farm -   Saving reader model to ../model/


In [24]:
finder = Finder(reader, retriever)

## Articles XML files

In [5]:
all_doc_files = glob.glob('../data/HACKXML0000000004/**/*.xml', recursive=True)[1:]

In [8]:
len(all_doc_files)

263

In [6]:
def nf2020toDict():
    paths = "../data/HACKXML0000000004/**/*.xml"
    target_tags = ['simple-para', 'para']

    docs = []
    for path in glob.glob(paths)[1:]:
        with open(path, 'r') as f: 
            data = BeautifulSoup(f.read() , "xml") 
        temp = {}
        temp["meta"] = {}
        temp["meta"]['title'] = data.find('title').getText()
        temp["meta"]["paper_id"] = data.find('aid').getText()
        temp["meta"]["doi"] = data.find('doi').getText()
        temp["meta"]["jid"] = data.find('jid').getText()
        
        paper_text = [t for t in data.find_all(text=True) if t.parent.name in target_tags]
        temp["text"] = ''.join(paper_text)
        
        docs.append(temp)
    return docs

In [7]:
nf_docs = nf2020toDict()

In [37]:
#nf_docs[1]['text']

In [26]:
#dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

In [8]:
# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(nf_docs)

11/12/2020 09:43:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.687s]


In [9]:
prediction = finder.get_answers(question="What is Neurofibromatosis", top_k_retriever=10, top_k_reader=5)

NameError: name 'finder' is not defined

In [16]:
print_answers(prediction, details="minimal")

[   {   'answer': 'benign neurofibroma',
        'context': 'is muscle in 2001. The pathological diagnoses were '
                   'lipofibroma or benign neurofibroma.When he presented at '
                   'our department in 2005, computed tomography'},
    {   'answer': 'subperiosteal neurofibromas',
        'context': 'e represents only the third case in the medical literature '
                   'of subperiosteal neurofibromas presenting with '
                   'subperiosteal hemorrhage. As in this case, t'},
    {   'answer': 'neurofibromatosis',
        'context': 'to be differentiated from a long list of diagnoses . '
                   'Historically, neurofibromatosis was the most common '
                   'mistaken diagnosis. The most important step i'},
    {   'answer': 'dermal neurofibroma',
        'context': 'ciated MPNST patients.Previously, we established a mouse '
                   'model of dermal neurofibroma (dNF) where we demonstrate '
                   

## Other paper docs only containing the abstract

In [10]:
ctf_hackathon_doc = pd.read_json('../data/ctf-hackathon-upload.json', lines=True)

In [11]:
ctf_hackathon_doc.columns

Index(['Eid', 'abstract', 'affiliation_organization', 'title', 'year',
       'sourcetitle', 'issn_print', 'doi', 'openaccess', 'meta_language',
       'References', 'pmid', 'publishername', 'keywords', 'funding_text'],
      dtype='object')

In [12]:
ctf_hackathon_doc.shape

(5702, 15)

In [13]:
len(ctf_hackathon_doc.doi.unique())

5129

In [14]:
with open('../data/HACKXML0000000004/dataset.xml', 'r') as f: 
    papers_info = f.read() 
papers_info_data = BeautifulSoup(papers_info, "xml") 

In [26]:
np.unique([tag.name for tag in papers_info_data.find_all()])

array(['aid', 'article-number', 'collection-title', 'dataset',
       'dataset-action', 'dataset-content', 'dataset-properties',
       'dataset-unique-ids', 'delayed-sponsored-article', 'doi',
       'dtd-version', 'embargo', 'files-info', 'filesize',
       'funding-body-id', 'issn', 'jid', 'jid-aid', 'journal-item',
       'journal-item-properties', 'journal-item-unique-ids', 'ml',
       'online-publication-date', 'pathname', 'pii', 'pit',
       'production-process', 'production-type', 'profile-code',
       'profile-dataset-id', 'purpose', 'sponsored-access', 'stage',
       'timestamp', 'type', 'version', 'version-number', 'weight'],
      dtype='<U25')

In [15]:
paper_xml_doi_ls = np.unique([t.getText() for t in papers_info_data.find_all('doi')])

In [16]:
# other nf docs
other_nfpaper_docs = ctf_hackathon_doc[~ctf_hackathon_doc.doi.isin(paper_xml_doi_ls)].copy()

In [17]:
other_nfpaper_docs.shape

(5440, 15)

In [38]:
other_nfpaper_docs.sample(5)

,Eid,abstract,affiliation_organization,title,year,sourcetitle,issn_print,doi,openaccess,meta_language,References,pmid,publishername,keywords,funding_text
2138,84872359373,Background context: An extraforaminal disc herniation may present as a retro...,"[[Department of Neurologic Surgery, Mayo Clinic], [Department of Radiology, ...",L5-S1 extraforaminal intraneural disc herniation mimicking a malignant perip...,2012,Spine Journal,[15299430],10.1016/j.spinee.2012.10.033,None,English,"[Tarlov, I.M., , Spinal perineurial and meningeal cysts. (1970), Journal of ...",23246211.0,Elsevier Inc.,"[Herniated disc, Lumbar, Malignant peripheral nerve sheath tumor, Retroperit...",NaN
3406,84897930288,Gastrointestinal stromal tumors (GIST) are the most common mesenchymatous ne...,"[[Servicios de Genética, Hospital Provincial Neuquén], [Servicios de Oncolog...","Dysphagia, melanosis, gastrointestinal stromal tumors and a germinal mutatio...",2014,Acta Gastroenterologica Latinoamericana,[03009033],NaN,NaN,English,"[Beghini, A., Tibiletti, M., Roversi, G., Chiaravalli, A., Serio, G., Capell...",24847623.0,Sociedad Argentina de Gastroenterologia,"[Dysphagia, GIST, KIT]",NaN
3995,85076330369,Background Neurofibromatosis type 1 (NF1) is a rare genetic disorder with us...,"[[Department of Psychology, University of Oslo], [Frambu Resource Centre for...",Health complaints and work experiences among adults with neurofibromatosis 1,2019,Occupational Medicine,[09627480],10.1093/occmed/kqz134,None,English,"[Abramowicz, A., Gos, M., , Neurofibromin in neurofibromatosis type 1 - muta...",31674646.0,Oxford University Press,"[Disability, NF1, Neurofibromatosis type 1, Rare disorders, Subjective healt...",NaN
3827,84923699181,Summary: Malignant peripheral nerve sheath tumour (MPNST) is a rare variety ...,"[[Department of Radiation Oncology, Dr BRA Institute Rotary Cancer Hospital,...",Malignant peripheral nerve sheath tumour of penis,2015,Andrologia,[03034569],10.1111/and.12267,None,English,"[Mortell, A., Amjad, B., Breatnach, F., Devaney, D., Puri, P., , Penile mali...",24635732.0,Blackwell Publishing Ltd,"[Malignant peripheral nerve sheath tumour, Multimodality, Penis, Treatment]",NaN
1843,84925385178,We present the first case of pediatric intracranial Mycobacterium abscessus ...,"[[Department of Pediatrics, Georgia Regents University, University of Georgi...",Intracranial Mycobacterium abscessus infection in a healthy toddler,2015,Journal of Pediatric Infectious Diseases,[13057707],10.1097/INF.0000000000000520,None,English,"[Kiliç, S., Tezcan, I., Sanal, O., Ersoy, F., , Common variable immunodefici...",25144796.0,IOS Press,"[Brain/pathology, Female, Humans, M. Abscessus, Mycobacterium infections/mic...",NaN


In [18]:
other_nfpaper_docs_dicts = []
for i, row in other_nfpaper_docs.iterrows():
    temp = {}
    if isinstance(row['abstract'], str):
        temp['text'] = row['abstract']
        temp['meta'] = {}
        temp['meta']['title'] = row['title']
        temp['meta']['doi'] = row['doi'] if isinstance(row['doi'], str) else ''
        #temp['meta']['References'] = ', '.join(row['References']) if isinstance(row['References'], list) else ''
        temp['meta']['keywords'] = ' '.join(row['keywords']) if isinstance(row['keywords'], list) and len(row['keywords'])>0 else ''
        other_nfpaper_docs_dicts.append(temp)

In [41]:
len(other_nfpaper_docs_dicts)

5018

In [42]:
other_nfpaper_docs_dicts[0]

{'text': "Introduction. Neurofibromatosis type 1 is an autosomal dominant disorder that occurs across all ethnic groups and affects approximately one in 4000 individuals. One of the most noticeable characteristics of the disease is the development of neurofibromas. Case presentation. A total of 258 patients (131 women, 127 men) with neurofibromatosis type 1 were evaluated between 1994 and 2004 in our hospital's dermatology department. Nine patients (3.45%, 95% confidence limits 1.22 to 5.68) had neurofibromas of the breast. One of these nine patients presented with an extensive congenital plexiform neurofibroma in the outer quadrants of her right breast, extending to the nipple-areolar complex. Meanwhile, three patients had more than one neurofibroma on the nipple-areolar complexes. Three patients had a family history of neurofibroma. Over the years 1994 to 2004, the cutaneous lesions were not associated with any malignancies. Presenting symptoms were related to conditions such as incr

In [19]:
document_store.get_document_count()

11/12/2020 09:46:36 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.019s]


262

In [20]:
document_store.write_documents(other_nfpaper_docs_dicts)

11/12/2020 09:46:49 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.807s]
11/12/2020 09:46:51 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.087s]
11/12/2020 09:46:52 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.038s]
11/12/2020 09:46:53 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.025s]
11/12/2020 09:46:54 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.039s]
11/12/2020 09:46:55 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.036s]
11/12/2020 09:46:56 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.003s]
11/12/2020 09:46:57 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.018s]


In [45]:
document_store.get_document_count()

11/07/2020 16:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.007s]


5280

In [46]:
other_nfpaper_docs.sample(4)['abstract']

2528    We report a case of synchronous multiple colon adenocarcinomas in a patient ...
5481                                                                                NaN
4221    Neurofibromin regulates cell motility via three distinct GTPase pathways act...
4120    Optic pathway gliomas (OPGs) occur in 1520 of children with neurofibromatosi...
Name: abstract, dtype: object

In [47]:
%timeit
prediction = finder.get_answers(question="Will NF1 turn into NF2", top_k_retriever=10, top_k_reader=5)

11/07/2020 16:55:46 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.031s]
11/07/2020 16:55:46 - INFO - haystack.retriever.sparse -   Got 10 candidates from retriever
11/07/2020 16:55:46 - INFO - haystack.finder -   Reader is looking for detailed answer in 12810 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.73s/ Batches]


In [102]:
print_answers(prediction, details="minimal")

[   {   'answer': 'There is no targeted drug therapy',
        'context': ' Purpose: There is no targeted drug therapy for NF2 '
                   'patients, and surgery or radiosurgery is not always '
                   'effective. Therefore, the exploration of new t'},
    {   'answer': 'This will generate new ideas about the biological '
                  'mechanisms involved in autism, which in turn may provide '
                  'new leads for the development of novel pharmaceutical '
                  'compounds.',
        'context': ' This will generate new ideas about the biological '
                   'mechanisms involved in autism, which in turn may provide '
                   'new leads for the development of novel pharmaceutical '
                   'compounds.'},
    {   'answer': 'neurofibromatosis type 2',
        'context': ' and researchers interested in neurofibromatosis type 1 '
                   '(NF1), neurofibromatosis type 2 (NF2), and schwannomatosis '
    

## Compiled PDF docs from the NF Registry website

In [21]:
document_store.get_document_count()

11/12/2020 09:47:29 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.010s]


5280

In [22]:
pdf_dicts = convert_files_to_dicts(dir_path='../data/nf_info_pdfs/', split_paragraphs=True)

11/12/2020 09:48:23 - INFO - haystack.preprocessor.utils -   Converting ../data/nf_info_pdfs/Genetics of NF - Children-s Tumor Foundation.pdf
11/12/2020 09:48:23 - INFO - haystack.preprocessor.utils -   Converting ../data/nf_info_pdfs/Schwannomatosis - Children-s Tumor Foundation.pdf
11/12/2020 09:48:23 - INFO - haystack.preprocessor.utils -   Converting ../data/nf_info_pdfs/CTF_About_NF2_Info_Sheet-2019.pdf
11/12/2020 09:48:23 - INFO - haystack.preprocessor.utils -   Converting ../data/nf_info_pdfs/NF1 - Children-s Tumor Foundation.pdf
11/12/2020 09:48:23 - INFO - haystack.preprocessor.utils -   Converting ../data/nf_info_pdfs/About_NF_120315.pdf
11/12/2020 09:48:23 - INFO - haystack.preprocessor.utils -   Converting ../data/nf_info_pdfs/NF2 - Children-s Tumor Foundation.pdf
11/12/2020 09:48:23 - INFO - haystack.preprocessor.utils -   Converting ../data/nf_info_pdfs/About_Schwannomatosis_Fact_Sheet_2020-2.pdf


In [23]:
for p in pdf_dicts:
    p['text'] = re.sub(r'For more information on.*', '', ' '.join(p['text'].split('\n')))

In [24]:
pdf_docs = [p for p in pdf_dicts if not p['text'].startswith('Help end NF by joining the confidential NF Registry') and len(p['text'])>50]

In [25]:
len(pdf_docs)

7

In [26]:
document_store.write_documents(pdf_docs)

11/12/2020 09:48:52 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.866s]


In [27]:
document_store.get_document_count()

11/12/2020 09:49:06 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.024s]


5287

In [43]:
nfregistry_docs = pd.DataFrame.from_dict(pdf_docs)
nfregistry_docs['document_identifie'] = nfregistry_docs['meta'].map(lambda x:x['name'].replace('.pdf', ''))
nfregistry_docs.rename(columns = {'text': 'document_text'}, inplace = True)
nfregistry_docs.drop(columns = 'meta', inplace=True)

In [44]:
nfregistry_docs

,document_text,document_identifie
0,Genetics of NF NF can occur in any family. Affected individuals can inhe fro...,Genetics of NF - Children-s Tumor Foundation
1,"Children's Tumor Foundat About Schwannomatosis Neurofibromatosis, also known...",Schwannomatosis - Children-s Tumor Foundation
2,"ABOUT NEUROFIBROMATOSIS TYPE 2 (NF2) Neurofibromatosis, or NF, is an under-...",CTF_About_NF2_Info_Sheet-2019
3,"Children's Tumor Foundation About NF1 Neurofibromatosis, also known as NF, i...",NF1 - Children-s Tumor Foundation
4,"N eurofibromatosis, or NF, is an under-recognized genetic disorder that can...",About_NF_120315
5,Common Signs of NF2 The appearance of the most common signs of NF2 usually v...,NF2 - Children-s Tumor Foundation
6,"ABOUT SCHWANNOMATOSIS Neurofibromatosis, or NF, is an under-recognized gene...",About_Schwannomatosis_Fact_Sheet_2020-2


In [45]:
nfregistry_docs.to_csv('../data/nfregistry_docs/nfregistry_info.csv')

## PMC Articles

In [28]:
pmc_papers = pd.read_csv("../data/pmc_papers.csv")

In [29]:
pmc_papers.head()

,pmc_id,title,authors,affliations,keywords,abstract
0,7643456,Discernment between candidate mechanisms for KRAS G13D colorectal cancer sen...,"Stites Edward C., Rossman Kent L., McFall Thomas, Schomburg Noah K.","grid.. Integrative Biology Laboratory, Salk Institute for Biological Studi...","KRAS, GTPase, EGFR, Cancer, Targeted therapy",Phase three clinical trial evidence suggests that colorectal cancers with th...
1,7643332,Psychometric properties of satisfaction with the childbirth education class ...,"Mirghafourvand Mojgan, Mohammad-Alizadeh-Charandabi Sakineh, Meedya Shahla, ...","grid..f Students’ Research Committee, Tabriz University of Medical science...","Satisfaction, Childbirth education, Validity, Reliability, Psychometric, Iran",Childbirth preparation classes can reduce pregnant women’s anxiety and fear ...
2,7641497,A Rare Incidence of Metachronous Neurovascular Lesions in a Child,"Chandrasekaran Deepak, Azariah Emmanuel D, Chinnaswami Ravindran, Subramani ...","Oral and Maxillofacial Surgery, Sri Ramachandra Institute of Higher Educatio...","mesenchymal tumor, solitary, intraosseous, juvenile, benign, hemimandibulect...",Neurofibroma is an uncommon benign tumor arising from nerve sheath fibroblas...
3,7640792,A Child with Enlarged Extremities – A Case of Macrodystrophia Lipomatosa,"Sundareswaran N, Gopinath G, Gunasekaran K","From the Department of Neurology, Government Mohan Kumaramangalam Medical Co...","Enlarged extremities, focal gigantism, macrodactyly, macrodystrophia lipomatosa","Macrodystrophia lipomatosa (ML) is a rare, non-hereditary, developmental ano..."
4,7609672,Primary pancreatic glomus tumor invading into the superior mesenteric vein: ...,"Sasaki Yu, Taki Yoshiro, Tamaki Ichiro, Sasano Hironobu, Komoto Izumi, Hosod...","grid..cDepartment of Surgery, Kansai Electric Power Hospital, Fukushima --, ...","Glomus tumor, Glomangiomyoma, Pancreas, Immunohistochemistry, Surgical resec...","Glomus tumors are subcutaneous tumors arising from glomus bodies, thermoregu..."


In [34]:
pmc_papers.shape

(10000, 6)

In [30]:
pmc_papers['authors'] = pmc_papers['authors'].map(lambda x: ', '.join(x.split(','))+' et al' if isinstance(x, str) and len(x.split(','))>3 else x)

In [31]:
pmc_papers_docs_dicts = []
for i, row in pmc_papers.iterrows():
    temp = {}
    if isinstance(row['abstract'], str):
        temp['text'] = row['abstract']
        temp['meta'] = {}
        temp['meta']['title'] = row['title']
        temp['meta']['authors'] = row['authors'] if isinstance(row['authors'], str) else ''
        #temp['meta']['affliations'] = row['affliations'] if isinstance(row['affliations'], str) else ''
        temp['meta']['pmc_id'] = row['pmc_id']
        temp['meta']['keywords'] = row['keywords'] if isinstance(row['keywords'], str) else ''
        pmc_papers_docs_dicts.append(temp)

In [32]:
document_store.write_documents(pmc_papers_docs_dicts)

11/12/2020 09:50:52 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.708s]
11/12/2020 09:50:53 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.044s]
11/12/2020 09:50:54 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.012s]
11/12/2020 09:50:55 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.043s]
11/12/2020 09:50:56 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.009s]
11/12/2020 09:50:57 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.020s]
11/12/2020 09:50:58 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.008s]
11/12/2020 09:51:00 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.020s]


In [35]:
document_store.get_document_count()

11/12/2020 09:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.101s]


14893

## QA model validation 

In [20]:
prediction = finder.get_answers(question="how many types of NF", top_k_retriever=10, top_k_reader=5)

11/09/2020 23:22:57 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.031s]
11/09/2020 23:22:57 - INFO - haystack.finder -   Got 10 candidates from retriever
11/09/2020 23:22:57 - INFO - haystack.finder -   Reader is looking for detailed answer in 18868 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.51 Batches/s]


In [21]:
print_answers(prediction, details="minimal")

[   {   'answer': 'three',
        'context': 'NF, is a genetic disorde that affects 1 in every 3,000 '
                   'people. There are three types NF: NF1, NF2, and '
                   'schwannomatosis. Neurofibromatosis type (NF1), '},
    {   'answer': 'two',
        'context': 'l-dominant disease with an incidence of 1/3,000 in live '
                   'births. There are two types of NF, NF 1 and NF 2, and NF 1 '
                   'is the most common type. This study'},
    {   'answer': 'two',
        'context': 'al-dominant disease with an incidence of 1 in 3000 live '
                   'births. There are two types of NF, NF 1 and NF 2, and NF 1 '
                   'is the most common. This study repo'},
    {   'answer': 'two or more neurofibromas of any type',
        'context': ' or 15 mm or larg post-pubertal individuals) Presence of '
                   'two or more neurofibromas of any type, or on more '
                   'plexiform neurofibromas Freckling under

In [141]:
prediction = finder.get_answers(question="what are some symtoms of NF2", top_k_retriever=10, top_k_reader=5)

10/13/2020 17:27:42 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.095s]
10/13/2020 17:27:42 - INFO - haystack.retriever.sparse -   Got 10 candidates from retriever
10/13/2020 17:27:42 - INFO - haystack.finder -   Reader is looking for detailed answer in 17740 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.20 Batches/s]


In [142]:
print_answers(prediction, details="minimal")

[   {   'answer': 'neurofibromatosis type 2',
        'context': 'r: (1) a malignant peripheral nerve sheath tumour (MPNST); '
                   '(2) neurofibromatosis type 2 (NF2); or (3) schwannomatosis '
                   '(SWNTS). The progressively enlar'},
    {   'answer': 'ocular, dermatological, and neurological symptoms',
        'context': 'as, children with NF2 most frequently present with ocular, '
                   'dermatological, and neurological symptoms. Arteriopathy, a '
                   'well-established feature in neur'},
    {   'answer': 'neurofibromatosis type 2',
        'context': 'for preoperative planning. QUESTION 6 How should patients '
                   'with neurofibromatosis type 2 (NF2) and vestibular '
                   'schwannoma be imaged and over what follow'},
    {   'answer': 'malformation of midline guideposts',
        'context': ' and prevent ectopic Slit2 expression. Loss of Nf2 causes '
                   'malformation of midline gui

In [17]:
prediction = finder.get_answers(question="how many mutations related to NF1", top_k_retriever=5, top_k_reader=3)

11/07/2020 17:34:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.030s]
11/07/2020 17:34:24 - INFO - haystack.finder -   Got 5 candidates from retriever
11/07/2020 17:34:24 - INFO - haystack.finder -   Reader is looking for detailed answer in 7989 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.71s/ Batches]


In [18]:
print_answers(prediction, details="minimal")

[   {   'answer': 'very similar mutations',
        'context': 'isease causing mutations." We show that the splicing '
                   'outcome of very similar mutations can be very different '
                   'due to variations in trans-acting factor('},
    {   'answer': 'KRAS mutants that strongly interacted with and '
                  'competitively inhibited NF1 drove wild-type RAS activation '
                  'in an EGFR-independent manner, whereas KRASG13D',
        'context': ' KRAS mutants that strongly interacted with and '
                   'competitively inhibited NF1 drove wild-type RAS activation '
                   'in an EGFR-independent manner, whereas KRASG13D'},
    {   'answer': 'neurofibromatosis type 1',
        'context': 'ts with tumours due to multiple endocrine neoplasia type 2 '
                   'and neurofibromatosis type 1 (NF1) showed similar '
                   'catecholamine metabolite and secretory pr'}]


In [23]:
prediction = finder.get_answers(question="what are the clinical manifestations for mutation Y489C in NF1 ", top_k_retriever=10, top_k_reader=5)

11/09/2020 23:28:05 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.091s]
11/09/2020 23:28:05 - INFO - haystack.finder -   Got 10 candidates from retriever
11/09/2020 23:28:05 - INFO - haystack.finder -   Reader is looking for detailed answer in 15015 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.53s/ Batches]


In [24]:
print_answers(prediction, details="minimal")

[   {   'answer': 'neurofibromas only and a combination of neurofibromas and '
                  'pigmentary changes',
        'context': 'linical manifestations are reported: neurofibromas only '
                   'and a combination of neurofibromas and pigmentary changes. '
                   'Segmental NF1 is an example of mosa'},
    {   'answer': 'intracranial hypotension',
        'context': ' They describe the clinical presentation and '
                   'manifestations of intracranial hypotension, as well as the '
                   'pertinent investigations. They also review the'},
    {   'answer': 'also had bilateral pheochromocytomas and mucosal neuromas',
        'context': 'was disease free after surgical resection. She also had '
                   'bilateral pheochromocytomas and mucosal neuromas leading '
                   'to the clinical diagnosis of MEN-2B. '},
    {   'answer': 'café-au-lait macules and skinfold freckling',
        'context': 'c criteria for ne

In [54]:
def get_answers(preds):
    all_answers = []
    for ap in preds['answers']:
        tmp = {}
        tmp['answer'] = ap.get('answer', '')
        tmp['context'] = ap.get('context', '')
        tmp['document_id'] = ap.get('document_id', '')
        meta_dict = ap.get('meta', None)
        if meta_dict:
            tmp['title'] = meta_dict.get('title', '')
            tmp['keywords'] = meta_dict.get('keywords', '')
        all_answers.append(tmp)
    pprint.pprint(all_answers)

In [25]:
print_answers(prediction)

{   'answers': [   {   'answer': 'neurofibromas only and a combination of '
                                 'neurofibromas and pigmentary changes',
                       'context': 'linical manifestations are reported: '
                                  'neurofibromas only and a combination of '
                                  'neurofibromas and pigmentary changes. '
                                  'Segmental NF1 is an example of mosa',
                       'document_id': 'a5d0c5ad-c426-445f-af68-7f899bc52657',
                       'meta': {   'doi': '10.2143/TVG.71.19.2001961',
                                   'keywords': '',
                                   'title': 'Segmental neurofibromatosis type '
                                            '1: A differential diagnostic '
                                            'challenge'},
                       'offset_end': 113,
                       'offset_end_in_doc': 277,
                       'offset_start': 37,
    

In [29]:
prediction = finder.get_answers(question="How many mutations discovered in NF1 ", 
                                top_k_retriever=10, 
                                top_k_reader=5
                                )

11/09/2020 23:30:51 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.039s]
11/09/2020 23:30:51 - INFO - haystack.finder -   Got 10 candidates from retriever
11/09/2020 23:30:51 - INFO - haystack.finder -   Reader is looking for detailed answer in 13427 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.14s/ Batches]


In [30]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Eleven',
        'context': 'ncing was used for analysis of family members of the '
                   'patients. Results: Eleven pathogenic variants were found '
                   'among the 11 patients, which included tw'},
    {   'answer': 'four',
        'context': 'associated with neurofibromatosis type 1 previously, while '
                   'the remaining four were discovered for the first time. '
                   'Prenatal diagnosis of two high-risk '},
    {   'answer': '10 out of 16 separate patient samples',
        'context': 'arate sets of samples. An NF1 mutation was discovered in '
                   '10 out of 16 separate patient samples. Our study provides '
                   'proof of principle that the sequenc'},
    {   'answer': 'six',
        'context': 'avorable cytogenetics and with monosomal karyotype '
                   'notably. We discovered six NF1 variants of unknown '
                   'significance in 7/20 patients of which onl

In [25]:
prediction = finder.get_answers(question="How often gliomas become malignant in NF1?", 
                                top_k_retriever=10, 
                                top_k_reader=5
                                )

11/10/2020 17:39:25 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.098s]
11/10/2020 17:39:25 - INFO - haystack.finder -   Got 10 candidates from retriever
11/10/2020 17:39:25 - INFO - haystack.finder -   Reader is looking for detailed answer in 17920 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.54 Batches/s]


In [26]:
# Question: "How often gliomas become malignant in NF1?"
print_answers(prediction, details="minimal")

[   {   'answer': '1.4%',
        'context': 'reading to hypothalamus. Spontaneous regression was '
                   'documented in 4/285 (1.4%) gliomas and the same number of '
                   'NF1 children died due to gliomas. Obstru'},
    {   'answer': 'majority of gliomas arising in adults with NF1 are '
                  'malignant cancers, typically glioblastoma, involving the '
                  'cerebral hemispheres',
        'context': 'trast, the majority of gliomas arising in adults with NF1 '
                   'are malignant cancers, typically glioblastoma, involving '
                   'the cerebral hemispheres. Our under'},
    {   'answer': 'About 15%',
        'context': 'elops in cells surrounding the optic nerve (the nerve '
                   'which controls v About 15% of children with NF1 will '
                   'develop an optic glioma, the majority of th'},
    {   'answer': 'never',
        'context': 'nign tumor may become malignant (cancerous), but m

In [34]:
doc = document_store.get_documents_by_id('7aae914f-58cb-45bf-89d2-267b8c1de4f5')

11/10/2020 22:08:48 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:1.227s]


In [37]:
doc[0].text

'Purpose: To evaluate the incidence and clinical importance of brain gliomas – optic pathway gliomas (OPGs) and especially gliomas outside the optic pathway (GOOP) for children with neurofibromatosis type 1 (NF1), additionally, to assess the causes of obstructive hydrocephalus in NF1 children with an emphasis on cases caused by idiopathic aqueduct stenosis. Subjects and methods: We analysed data from 285 NF1 children followed up on our department from 1990 to 2010 by the same examination battery. Results: We have found OPGs in 77/285 (27%) children and GOOPs in 29/285 (10,2%) of NF1 children, of who 19 had OPG and GOOP together, so the total number of brain glioma was 87/285 (30,5%). GOOPs were significantly more often treated than OPGs (p > 0.01). OPGs contain clinically important subgroup of 14/285 (4.9%) spreading to hypothalamus. Spontaneous regression was documented in 4/285 (1.4%) gliomas and the same number of NF1 children died due to gliomas. Obstructive hydrocephalus was found

In [38]:
prediction = finder.get_answers(question="What is the likelihood for mutation R2429X to be pathogenic?", 
                                top_k_retriever=10, 
                                top_k_reader=5
                                )

11/11/2020 08:01:02 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.728s]
11/11/2020 08:01:02 - INFO - haystack.finder -   Got 10 candidates from retriever
11/11/2020 08:01:02 - INFO - haystack.finder -   Reader is looking for detailed answer in 15232 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.42s/ Batches]


In [39]:
# Question: "What is the likelihood for mutation R2429X to be pathogenic?"
print_answers(prediction, details="minimal")

[   {   'answer': 'lower burden',
        'context': ' elastosis, infrequently harbored TERT promoter mutations, '
                   'and had a lower burden of pathogenic mutations than '
                   'spitzoid melanomas with non-Spitz MAPK '},
    {   'answer': '80.9%',
        'context': 'nal Institute of Health NF1 criteria, a pathogenic '
                   'mutation was found in 80.9% of these patients. '
                   'Seventy-four percent of the index patients with an N'},
    {   'answer': 'increases with age',
        'context': 'o 2%. The likelihood an individual is harboring a SPRED1 '
                   'mutation increases with age if multiple, nonpigmentary NF1 '
                   'findings are absent. Legius syndro'},
    {   'answer': 'as low as 0.05 annually',
        'context': 'S. The likelihood of this occurring in NF2 was considered '
                   'to be as low as 0.05 annually. None of 28 families with '
                   'familial unilateral VS

In [40]:
#Do NF1 kids develop autism
prediction = finder.get_answers(question="Do NF1 kids develop autism", 
                                top_k_retriever=10, 
                                top_k_reader=5
                                )

11/11/2020 08:09:44 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.115s]
11/11/2020 08:09:44 - INFO - haystack.finder -   Got 10 candidates from retriever
11/11/2020 08:09:44 - INFO - haystack.finder -   Reader is looking for detailed answer in 14616 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.82s/ Batches]


In [55]:
get_answers(prediction)

[{'answer': 'Nf1 +/– mice exhibited both increased USVs across development and '
            'alterations in aspects of pitch, while Nf1 GFAP CKO mice '
            'demonstrated a decrease in USVs. Developmental milestones, such '
            'as weight, pinnae detachment, and eye opening, were not disrupted '
            'in either model, indicating the USV deficits were not due to '
            'gross developmental delay',
  'context': ' Nf1 +/– mice exhibited both increased USVs across development '
             'and alterations in aspects of pitch, while Nf1 GFAP CKO mice '
             'demonstrated a decrease in USVs. Developmental milestones, such '
             'as weight, pinnae detachment, and eye opening, were not '
             'disrupted in either model, indicating the USV deficits were not '
             'due to gross developmental delay',
  'document_id': 'e078992d-d253-4847-b37a-4a6320735132',
  'keywords': 'autism communication mouse neurofibromatosis type 1 ultrasoni